In [2]:
import requests

In [3]:
from bs4 import BeautifulSoup

In [4]:
import pandas as pd

In [5]:
books = []

def extract_ingredients(product_url):
    response = requests.get(product_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        ingredients_element = soup.find('div', class_='athenaProductPageSynopsisContent')
        ingredients = ingredients_element.text.strip() if ingredients_element else ""
        return ingredients
    else:
        print(f"Failed to retrieve product page. Status code: {response.status_code}")
        return ""

for i in range(1, 23):
    url = f"https://www.lookfantastic.com/health-beauty/new/trending.list?pageNumber={i}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract data from the current page
        product_blocks = soup.select('.productBlock')
        for block in product_blocks:
            # Extracting product information
            product_name = block.select_one('.productBlock_productName').text.strip()
            product_brand = block.select_one('[data-product-brand]').get('data-product-brand')
            product_price = block.select_one('[data-product-price]').get('data-product-price')

            # Extracting product URL
            product_url = block.select_one('.productBlock_link').get('href')
            full_product_url = f"https://www.lookfantastic.com{product_url}"

            # Extracting ingredients using the product URL
            ingredients = extract_ingredients(full_product_url)

            # Append data to books list
            books.append([product_name, product_brand, product_price, ingredients])

    except requests.RequestException as e:
        print(f"Failed to retrieve page {i}. Exception: {e}")


In [6]:
df = pd.DataFrame(books, columns=['Product_name','Product_brand','Product_price','ingredients'])

In [7]:
df.to_csv('shop.csv')